In [1]:
# main.ipynb content to be used in Jupyter Notebook format

# Section 1: Setup
import torch
import pandas as pd
from torch.utils.data import DataLoader
from src.adm_prednet.stacked_model import ADMStackedModel
from src.adm_prednet.pe_utils import add_temporal_pe, add_spatial_pe
from src.adm_prednet.masked_loss import masked_mse
from src.adm_prednet.evaluate import evaluate_model
from src.adm_prednet.save_predictions import save_predictions_to_csv
from src.adm_prednet.visualize_utils import plot_comparison
from src.grid_utils import create_gridded_data
from src.data_utils import split_data_chronologically
# Section 4: Dataset Class from Grid
from torch.utils.data import Dataset
class GriddedSeq2SeqDataset(Dataset):
    def __init__(self, gridded_tensor, input_steps=12, target_steps=1, target_indices=[0, 1]):
        self.data = torch.tensor(gridded_tensor, dtype=torch.float32)
        self.input_steps = input_steps
        self.target_steps = target_steps
        self.target_indices = target_indices
        self.grid_shape = self.data.shape[1:3]  # (H, W)

    def __len__(self):
        return self.data.shape[0] - self.input_steps - self.target_steps + 1

    def __getitem__(self, idx):
        X = self.data[idx:idx + self.input_steps]  # [T, H, W, C]
        X = X.permute(0, 3, 1, 2).float()           # → [T, C, H, W]
        Y = self.data[idx + self.input_steps]      # [H, W, C]
        Y = Y[..., self.target_indices].permute(2, 0, 1)  # [C_target, H, W]
        return X, Y

        




In [2]:
import yaml
import torch
import pandas as pd
from torch.utils.data import DataLoader, ConcatDataset
import optuna
import os
from src.adm_prednet.train import train_adm_model
from src.adm_prednet.stacked_model import ADMStackedModel
from src.adm_prednet.evaluate import evaluate_model
from src.adm_prednet.save_predictions import save_predictions_to_csv
from pathlib import Path
from src.data_utils import split_data_chronologically
from src.grid_utils import create_gridded_data
class GriddedSeq2SeqDataset(Dataset):
    def __init__(self, gridded_tensor, input_steps=12, target_steps=1, target_indices=[0, 1]):
        self.data = torch.tensor(gridded_tensor, dtype=torch.float32)
        self.input_steps = input_steps
        self.target_steps = target_steps
        self.target_indices = target_indices
        self.grid_shape = self.data.shape[1:3]  # (H, W)

    def __len__(self):
        return self.data.shape[0] - self.input_steps - self.target_steps + 1

    def __getitem__(self, idx):
        X = self.data[idx:idx + self.input_steps]  # [T, H, W, C]
        X = X.permute(0, 3, 1, 2).float()           # → [T, C, H, W]
        Y = self.data[idx + self.input_steps]      # [H, W, C]
        Y = Y[..., self.target_indices].permute(2, 0, 1)  # [C_target, H, W]
        return X, Y

class ADMForecastingPipeline:
    def __init__(self, config_path):
        self.config = yaml.safe_load(open(config_path))
        self.model = None
        self.land_mask = None
        self.raw_df = None
        self.study = None
        self.best_params = None

    def load_data(self):
        self.full_df = pd.read_csv(self.config["data"]["csv_path"])
        self.raw_df = self.full_df  # For saving predictions

        self.train_df, self.val_df, self.test_df = split_data_chronologically(self.full_df, self.config)
        self.gridded_tensor_train, self.land_mask = create_gridded_data(self.train_df, self.config)  # assume same shape for all
        self.gridded_tensor_val, self.land_mask = create_gridded_data(self.val_df, self.config)  # assume same shape for all
        self.gridded_tensor_test, self.land_mask = create_gridded_data(self.test_df, self.config)  # assume same shape for all
        self.gridded_tensor_full, self.land_mask = create_gridded_data(self.full_df, self.config)  # assume same shape for all
        self.train_dataset = GriddedSeq2SeqDataset(
            self.gridded_tensor_train,
            input_steps=self.config['training']['input_steps'],
            target_steps=self.config["training"]["output_steps"],
            target_indices=list(range(len(self.config["model"]["output_targets"])))
        )
        self.val_dataset = GriddedSeq2SeqDataset(
            self.gridded_tensor_val,
            input_steps=self.config['training']['input_steps'],
            target_steps=self.config["training"]["output_steps"],
            target_indices=list(range(len(self.config["model"]["output_targets"])))
        )
        self.test_dataset = GriddedSeq2SeqDataset(
            self.gridded_tensor_test,
            input_steps=self.config['training']['input_steps'],
            target_steps=self.config["training"]["output_steps"],
            target_indices=list(range(len(self.config["model"]["output_targets"])))
        )
        self.full_dataset = GriddedSeq2SeqDataset(
            self.gridded_tensor_full,
            input_steps=self.config['training']['input_steps'],
            target_steps=self.config["training"]["output_steps"],
            target_indices=list(range(len(self.config["model"]["output_targets"])))
        )
        self.land_mask =  torch.tensor(self.land_mask, dtype=torch.float32).cuda()
        Path(self.config["output"]["predictions_csv"]).mkdir(parents=True, exist_ok=True)

    def build_model(self):
        self.model = ADMStackedModel(
            input_channels=self.config["model"]["input_channels"],
            hidden_channels=self.config["model"]["hidden_channels"],
            n_layers=self.config["model"]["n_layers"],
            output_targets=self.config["model"]["output_targets"]
        ).cuda()

    def train(self):
        self.study = optuna.create_study(direction="minimize")
        self.study.optimize(lambda trial: self.objective(trial, self.config, self.train_dataset, self.val_dataset, self.land_mask), n_trials=10)

        self.best_params = self.study.best_trial.params
        print("Best hyperparameters found by Optuna:", self.best_params)

        # Reconstruct hidden_channels
        n_layers = self.best_params['n_layers']
        hidden_channels = [self.best_params[f'hidden_channels_{i}'] for i in range(n_layers)]
        print("Reconstructed hidden_channels:", hidden_channels)

        cfg_final = self.config.copy()
        cfg_final['model']['hidden_channels'] = hidden_channels
        cfg_final['model']['n_layers'] = n_layers
        cfg_final['training']['learning_rate'] = self.best_params['learning_rate']
        cfg_final['training']['dropout_rate'] = self.best_params['dropout_rate']
        cfg_final['training']['batch_size'] = self.best_params['batch_size']

        self.train_val_dataset = ConcatDataset([self.train_dataset, self.val_dataset])
        train_val_loader = DataLoader(self.train_val_dataset, batch_size=cfg_final['training']['batch_size'], shuffle=True)
        test_loader = DataLoader(self.test_dataset, batch_size=cfg_final['training']['batch_size'], shuffle=False)

        self.model = train_adm_model(train_val_loader, cfg_final, 
        val_loader=test_loader, land_mask=self.land_mask,log_path=self.config["output"]["predictions_csv"]+"prediction_log.csv")
        torch.save(self.model.state_dict(), "final_model.pth")

        # Final evaluation
        metrics = evaluate_model(self.model, test_loader, self.land_mask, output_targets=cfg_final['model']['output_targets'])
        print("Final Evaluation on Test Set:")
        for target, vals in metrics.items():
            print(f"  {target}: {vals}")

    def evaluate(self):
        val_loader = DataLoader(self.val_dataset, batch_size=self.config["training"]["batch_size"], shuffle=False)
        metrics = evaluate_model(self.model, val_loader, self.land_mask, self.config["model"]["output_targets"])
        print("Evaluation Metrics on Validation Set:")
        for target, vals in metrics.items():
            print(f"  {target}: {vals}")
        return metrics

    def predict_and_save(self):
        val_loader = DataLoader(self.val_dataset, batch_size=self.config["training"]["batch_size"], shuffle=False)
        save_predictions_to_csv(
            self.model,
            val_loader,
            self.land_mask,
            self.config["model"]["output_targets"],
            self.raw_df,
            self.config["output"]["predictions_csv"]
        )

    def objective(self, trial, config, train_dataset, val_dataset, land_mask):

        n_layers = trial.suggest_int('n_layers', 2, 5)
        hidden_channels = [trial.suggest_categorical(f'hidden_channels_{i}', [128, 256,512]) for i in range(n_layers)]
        learning_rate = trial.suggest_float('learning_rate', 2e-5, 5e-3, log=True)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.3)
        batch_size = trial.suggest_int('batch_size', 8, 32, step=8)

        config['model']['hidden_channels'] = hidden_channels
        config['model']['n_layers'] = n_layers
        config['training']['learning_rate'] = learning_rate
        config['training']['dropout_rate'] = dropout_rate
        config['training']['batch_size'] = batch_size

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = train_adm_model(train_loader, config, val_loader=val_loader, land_mask=land_mask)
        metrics = evaluate_model(model, val_loader, land_mask, output_targets=config['model']['output_targets'])
        ret_error = []
        for k,v in metrics.items():
            ret_error.append(v['RMSE'])
        return sum(ret_error) / len(ret_error)

    def run(self):
        self.load_data()
        self.train()
        self.evaluate()
        self.predict_and_save()


c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.1 is exactly one major version older than the runtime version 6.31.1 at api.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pipeline_prepet = ADMForecastingPipeline("config_prepet.yaml")
pipeline_prepet.run()



Splitting data: Train ends 2017-12-31 00:00:00, Validation ends 2020-12-31 00:00:00
Train set shape: (251316, 19), Time range: 1901-01-16 00:00:00 to 2017-12-16 00:00:00
Validation set shape: (6444, 19), Time range: 2018-01-16 00:00:00 to 2020-12-16 00:00:00
Test set shape: (6444, 19), Time range: 2021-01-16 00:00:00 to 2023-12-16 00:00:00
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (1404, 29, 16, 15)...


c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_idx'] = ((df[lat_col] - lat_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col_idx'] = ((df[lon_col] - lon_min) / fixed_step).round().astype(int)


--- Data Gridding Process Finished ---
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (36, 29, 16, 15)...
--- Data Gridding Process Finished ---
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (36, 29, 16, 15)...


c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_idx'] = ((df[lat_col] - lat_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col_idx'] = ((df[lon_col] - lon_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

--- Data Gridding Process Finished ---
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (1476, 29, 16, 15)...
--- Data Gridding Process Finished ---


[I 2025-06-17 01:01:47,528] A new study created in memory with name: no-name-d523ff0c-84ff-4ce5-ac5a-53ed929bd37e
[W 2025-06-17 01:02:12,864] Trial 0 failed with parameters: {'n_layers': 2, 'hidden_channels_0': 128, 'hidden_channels_1': 64, 'learning_rate': 4.4744793387882014e-05, 'dropout_rate': 0.22261005343030205, 'batch_size': 16} because of the following error: RuntimeError('Parent directory checkpoints/adm_prednet/predictions_prepet does not exist.').
Traceback (most recent call last):
  File "C:\Users\peera\AppData\Roaming\Python\Python312\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\peera\AppData\Local\Temp\ipykernel_48624\2952359607.py", line 87, in <lambda>
    self.study.optimize(lambda trial: self.objective(trial, self.config, self.train_dataset, self.val_dataset, self.land_mask), n_trials=1)
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1 - Train Loss: 0.1619
Epoch 1 - Validation Loss: 0.0655


RuntimeError: Parent directory checkpoints/adm_prednet/predictions_prepet does not exist.

In [4]:
pipeline_all = ADMForecastingPipeline("config_prepetspei.yaml")
pipeline_all.run()

Splitting data: Train ends 2017-12-31 00:00:00, Validation ends 2020-12-31 00:00:00
Train set shape: (251316, 19), Time range: 1901-01-16 00:00:00 to 2017-12-16 00:00:00
Validation set shape: (6444, 19), Time range: 2018-01-16 00:00:00 to 2020-12-16 00:00:00
Test set shape: (6444, 19), Time range: 2021-01-16 00:00:00 to 2023-12-16 00:00:00
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (1404, 29, 16, 16)...


c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_idx'] = ((df[lat_col] - lat_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col_idx'] = ((df[lon_col] - lon_min) / fixed_step).round().astype(int)


--- Data Gridding Process Finished ---
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (36, 29, 16, 16)...
--- Data Gridding Process Finished ---
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (36, 29, 16, 16)...
--- Data Gridding Process Finished ---
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees


c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_idx'] = ((df[lat_col] - lat_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col_idx'] = ((df[lon_col] - lon_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (1476, 29, 16, 16)...


[I 2025-06-17 01:05:51,982] A new study created in memory with name: no-name-f90d33fe-2bb0-45a7-b73f-197c23ebb44f


--- Data Gridding Process Finished ---
Epoch 1 - Train Loss: 0.1475
Epoch 1 - Validation Loss: 0.0594


[I 2025-06-17 01:06:19,097] Trial 0 finished with value: 0.26934635639190674 and parameters: {'n_layers': 2, 'hidden_channels_0': 128, 'hidden_channels_1': 128, 'learning_rate': 7.719043349275802e-05, 'dropout_rate': 0.29505529941179803, 'batch_size': 8}. Best is trial 0 with value: 0.26934635639190674.


Best hyperparameters found by Optuna: {'n_layers': 2, 'hidden_channels_0': 128, 'hidden_channels_1': 128, 'learning_rate': 7.719043349275802e-05, 'dropout_rate': 0.29505529941179803, 'batch_size': 8}
Reconstructed hidden_channels: [128, 128]
Epoch 1 - Train Loss: 0.1396
Epoch 1 - Validation Loss: 0.0505
Training log saved to run_outputs/adm_prednet/predictions_all/prediction_log.csv
Final Evaluation on Test Set:
  PET: {'RMSE': np.float32(0.3586714), 'MAE': np.float32(0.2858248), 'R2': 0.7818723917007446}
  PRE: {'RMSE': np.float32(0.2398288), 'MAE': np.float32(0.18879706), 'R2': 0.805541455745697}
  SPEI: {'RMSE': np.float32(0.14843479), 'MAE': np.float32(0.11828766), 'R2': 0.8967832326889038}
Evaluation Metrics on Validation Set:
  PET: {'RMSE': np.float32(0.3254588), 'MAE': np.float32(0.26095405), 'R2': 0.8202466368675232}
  PRE: {'RMSE': np.float32(0.26462525), 'MAE': np.float32(0.20814984), 'R2': 0.8123681545257568}
  SPEI: {'RMSE': np.float32(0.17962982), 'MAE': np.float32(0.1391